In [86]:
%load_ext autoreload
%autoreload 2

import os
import oddsapi
import requests
import joblib
import pandas as pd
import numpy as np
from common.utils.odds_utils import get_tournament_winner_odds, get_group_stage_odds, get_group_stage_OU
from common.utils.holdet_utils import get_holdet_data, get_play_off_opps, get_rw
from common.utils.simulation_utils import WorldCupTeam, WorldCupSim
from common.utils.group_stage import get_group_stage, playoffs
from common.utils.dict_utils import eng2dan
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
ROUND = 3
API_KEY = os.getenv("API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
# load data
df_holdet = get_holdet_data()
# df_winner = get_tournament_winner_odds(API_KEY)
# df_group_stage = get_group_stage_odds(API_KEY)
# df_get_group_stage_OU = get_group_stage_OU(API_KEY)
# df_groups = pd.read_csv("common/ressources/df_groups.csv")[["Team","group"]]
# df_group_stage = df_group_stage[['home_team','away_team','prob1','probx', 'prob2']].merge(df_groups.rename(columns =  {"Team":"home_team"}), on = "home_team")

In [63]:
_df_group_stage = get_group_stage(df_get_group_stage_OU.copy(), df_group_stage.copy(), ROUND, eng2dan)

In [64]:
_df_group_stage = _df_group_stage.iloc[1:]

In [88]:
df_rw = get_rw(df_holdet)
df_rw.rename(columns = {"Player_match": "name1"}, inplace = True)
print(df_holdet.shape)
df_holdet = df_holdet.merge(df_rw, on = "name1", how ="left")
print(df_holdet.shape)
df_holdet = df_holdet[['name1', 'name2', 'country', 'price', 'position', 'Player', 'Team', 'Opp', 'Pos', 'MIN','G', 'A', 'SOG', 'Y','R']]
print(df_holdet.shape)

check names, not all is found
(260, 10)
(262, 39)
(262, 15)


In [89]:
round = 1
df_temp1 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[:,["home_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"home_team":"country"}), on = "country")
df_temp1.rename(columns = {"prob1":f"prob_winning_{round}","probx":f"prob_draw_{round}","prob2":f"prob_lose_{round}","xGH":f"E_goals_{round}","xGA":f"E_goals_against_{round}"}, inplace = True)

df_temp2 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[:,["away_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"away_team":"country"}), on = "country")
df_temp2.rename(columns = {"prob1":f"prob_lose_{round}","probx":f"prob_draw_{round}","prob2":f"prob_winning_{round}","xGA":f"E_goals_{round}","xGH":f"E_goals_against_{round}"}, inplace = True)

df_temp_temp = pd.concat([df_temp1,df_temp2],axis=0)
df_holdet = df_holdet.merge(df_temp_temp, on = ["name1","name2","country"])
df_holdet["selected"] = ~df_holdet['G'].isna()

In [90]:
i=1
for country in df_holdet["country"].unique():
    xG_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float).sum()
    xA_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float).sum()
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float)/xG_sum
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float)/xA_sum

    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_total"] = -5000
    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_round1"] = -5000
    for player in df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),"name1"]:
        points = 0
        goal_share = df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]
        assist_share = df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]
        yellow_prob = df_holdet.loc[df_holdet["name1"]==player,'Y'].astype(float).values[0]
        red_prob = df_holdet.loc[df_holdet["name1"]==player,'R'].astype(float).values[0]
        points += 7000
        points += df_holdet.loc[df_holdet["name1"]==player,f'prob_winning_{i}'].values[0]*25000
        points += df_holdet.loc[df_holdet["name1"]==player,f'prob_draw_{i}'].values[0]*5000
        points += df_holdet.loc[df_holdet["name1"]==player,f'prob_lose_{i}'].values[0]*-15000
        points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*10000
        points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0]*-10000
        points += yellow_prob * -20000
        points += yellow_prob*yellow_prob*-20000
        points += red_prob * -50000
        
        if df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Mål":
            points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*75000
            
        elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Forsvar":
            points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*50000
            points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*175000
            points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000
            
        elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Midtbane":
            points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*150000
            points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000
            
        elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Angreb":
            points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*125000
            points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000
            
        df_holdet.loc[df_holdet["name1"]==player,"E_points_round1"] = points
                
        df_holdet.loc[df_holdet["name1"]==player,"E_points_total"] = points

In [91]:
df_holdet.sort_values("E_points_total", ascending = False)

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,...,prob_winning_1,prob_draw_1,prob_lose_1,E_goals_1,E_goals_against_1,selected,G_share,A_share,E_points_total,E_points_round1
2,Neymar Jr.,N. Jr.,Brasilien,9364000.0,Angreb,Neymar,BRA,KOR,F/M,90,...,0.711898,0.186189,0.101914,2.221022,0.317957,True,0.291498,0.243094,152305.260981,152305.260981
3,Lionel Messi,L. Messi,Argentina,8695000.0,Angreb,Lionel Messi,ARG,AUS,F/M,90,...,0.427870,0.304857,0.267272,1.924371,1.202073,True,0.348548,0.385057,150328.432212,150328.432212
110,Raphael Guerreiro,R. Guerreiro,Portugal,3972000.0,Forsvar,Raphael Guerreiro,POR,SUI,M/D,90,...,0.594059,0.243564,0.162376,2.231144,0.609846,True,0.140127,0.134615,135479.924844,135479.924844
57,Joao Felix,J. Felix,Portugal,4890000.0,Angreb,Joao Felix,POR,SUI,F/M,90,...,0.594059,0.243564,0.162376,2.231144,0.609846,True,0.248408,0.211538,126892.177138,126892.177138
8,Bruno Fernandes,B. Fernandes,Portugal,7700000.0,Midtbane,Bruno Fernandes,POR,SUI,F/M,90,...,0.594059,0.243564,0.162376,2.231144,0.609846,True,0.210191,0.173077,125561.215874,125561.215874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,Luka Modric,L. Modric,Kroatien,5193000.0,Midtbane,Luka Modric,CRO,JPN,M,90,...,0.101914,0.186189,0.711898,0.317957,2.221022,True,0.099379,0.173077,-14126.739237,-14126.739237
161,Mateo Kovacic,M. Kovacic,Kroatien,3107000.0,Midtbane,Mateo Kovacic,CRO,JPN,M,90,...,0.101914,0.186189,0.711898,0.317957,2.221022,True,0.037267,0.115385,-20333.689425,-20333.689425
125,Marcelo Brozovic,M. Brozovic,Kroatien,3647000.0,Midtbane,Marcelo Brozovic,CRO,JPN,M,90,...,0.101914,0.186189,0.711898,0.317957,2.221022,True,0.074534,0.057692,-25196.910847,-25196.910847
189,Sofyan Amrabat,S. Amrabat,Marokko,2625000.0,Midtbane,Sofyan Amrabat,MAR,ESP,M,90,...,0.162376,0.243564,0.594059,0.609846,2.231144,True,0.020619,0.000000,-30160.523258,-30160.523258


In [92]:
df_holdet_old = pd.read_csv("common/ressources/df_holdet_old")

In [93]:
df_holdet_old["E_points_total"] = -1000000

In [94]:
df_holdet = pd.concat([df_holdet, df_holdet_old[df_holdet.columns]], axis=0)
df_holdet.sort_values("E_points_total", ascending = False, inplace=True)
df_holdet.drop_duplicates(subset = "name1", keep="first", inplace = True)

In [97]:
df_holdet.sort_values("E_points_total",ascending=False)

In [98]:
df_holdet.to_csv("common/ressources/df_holdet", index = False)

In [29]:
df_holdet.loc[df_holdet["country"]=="Senegal","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Ghana","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Japan","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Serbien","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Wales","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Senegal","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Ghana","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Japan","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Serbien","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Wales","E_points_round1"] = 0

In [15]:
df_holdet.sort_values("E_points_total", ascending = False).iloc[0:40]

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,...,xG_Final,xGa_R16,xGa_QF,xGa_SF,xGa_Final,selected,G_share,A_share,E_points_total,E_points_round1
0,Kylian Mbappe,K. Mbappe,Frankrig,9282000.0,Angreb,Kylian Mbappe,FRA,AUS,F,90.0,...,1.593750,0.367617,0.594549,0.483353,0.367617,True,0.354167,0.229167,247019.777675,158008.516822
77,Lionel Messi,L. Messi,Argentina,8131000.0,Angreb,Lionel Messi,ARG,KSA,F/M,90.0,...,1.920209,0.575312,0.533931,0.408978,0.408978,True,0.318339,0.232804,242046.078613,160925.202129
207,Robert Lewandowski,R. Lewandowski,Polen,6482000.0,Angreb,Robert Lewandowski,POL,MEX,F,90.0,...,0.090019,0.518865,0.223467,0.065375,0.029840,True,0.625000,0.184466,234414.335680,223745.799464
108,Cody Gakpo,C. Gakpo,Holland,4730000.0,Angreb,Cody Gakpo,NED,SEN,F/M,90.0,...,0.500893,0.133422,0.160107,0.242267,0.160107,True,0.262443,0.286585,234277.793622,135679.584228
51,Neymar Jr.,N. Jr.,Brasilien,9062000.0,Angreb,Neymar,BRA,SRB,F/M,90.0,...,1.766713,0.288051,0.253385,0.217046,0.217046,True,0.282297,0.227586,207527.430854,136237.481856
25,Harry Kane,H. Kane,England,9194000.0,Angreb,Harry Kane,ENG,IRN,F,90.0,...,1.520202,0.519206,0.318721,0.420607,0.318721,True,0.290456,0.149425,204010.560354,132442.922911
104,Denzel Dumfries,D. Dumfries,Holland,5602000.0,Forsvar,Denzel Dumfries,NED,SEN,M/D,90.0,...,0.500893,0.133422,0.160107,0.242267,0.160107,True,0.099548,0.146341,197384.894336,111818.451542
155,Kevin De Bruyne,K. De Bruyne,Belgien,7585000.0,Midtbane,Kevin De Bruyne,BEL,CAN,F/M,90.0,...,0.848258,1.728358,0.226684,0.533374,0.533374,True,0.211618,0.471264,193261.571767,139706.675599
78,Lautaro Martinez,L. Martinez,Argentina,7486000.0,Angreb,Lautaro Martinez,ARG,KSA,F,90.0,...,1.920209,0.575312,0.533931,0.408978,0.408978,True,0.273356,0.084656,190247.555644,126369.544641
267,Marco Asensio,M. Asensio,Spanien,4722000.0,Midtbane,Marco Asensio,ESP,CRC,F/M,90.0,...,1.346297,0.714120,0.287064,0.534727,0.534727,True,0.301818,0.210227,176851.067935,94328.034892
